# Latihan 4: Manajemen Pengguna

Latihan berikut bertujuan untuk memberikan gambaran mengenai bagaimana administrator dapat berinteraksi dengan pengguna portal melalui Notebook

## Mengakses informasi pengguna


In [1]:
from arcgis.gis import GIS
gis = GIS("Home")

Untuk mengakses informasi pengguna, dapat digunakan objek `users`

In [2]:
me = gis.users.me
me

<User username:Dany_Laksono_LearnArcGIS7>


## Properti object 'user'
You can query much more information about the user account as properties on the `User` object:

In [3]:
me.access

'org'

You can find out when an account was last active and determine if an account was abandoned and remove it if necessary.

In [4]:
import time
# convert Unix epoch time to local time
created_time = time.localtime(me.created/1000)
print("Created: {}/{}/{}".format(created_time[0], created_time[1], created_time[2]))

last_accessed = time.localtime(me.lastLogin/1000)
print("Last active: {}/{}/{}".format(last_accessed[0], last_accessed[1], last_accessed[2]))

Created: 2020/8/22
Last active: 2020/9/1


Let us print some more information about this account

In [5]:
print(me.description, " ", me.email, " ", me.firstName, " ", me.lastName, " ", me.fullName)
print(me.level, " ", me.mfaEnabled, " ", me.provider, " ", me.userType)

AttributeError: 'User' object has no attribute 'firstName'

You can determine how much storage is being used by this account

In [ ]:
quota = me.storageQuota
used = me.storageUsage
pc_usage = round((used / quota)*100, 2)
print("Usage: " + str(pc_usage) + "%")

You can determine the groups the user is a member of:

In [ ]:
user_groups = me.groups
print("Member of " + str(len(user_groups)) + " groups")

# groups are returned as a dictionary. Lets print the first dict as a sample
user_groups[0]


## Mencari akun pengguna
The `search()` method of `UserManager` class helps you search for users of the org. The `query` parameter in the `search()` method accepts standard [ArcGIS REST API queries](https://developers.arcgis.com/rest/users-groups-and-items/search-reference.htm) and behaves similar to the search method on `ContentManager` and `GroupManager` classes. To illustrate this better, let us search ArcGIS Online as there are many more users available there.

In [ ]:
# anonymous connection to ArcGIS Online
ago_gis = GIS()

In [ ]:
# search the users whose email address ends with esri.com
esri_public_accounts = ago_gis.users.search(query='email = @esri.com')
len(esri_public_accounts)

Each element in the list returned is a `User` object that you can query.

In [ ]:
# lets filter out Esri curator accounts from this list
curator_accounts = [acc for acc in esri_public_accounts if acc.username.startswith('Esri_Curator')]
curator_accounts

In [ ]:
curator_accounts[0]

Once you know a user's username, you can access that object using the **`get()`** method. Let us access the Esri curator account for historical maps

In [ ]:
esri_hist_maps = ago_gis.users.get(username='Esri_Curator_Historical')
esri_hist_maps


## Membuat akun pengguna baru
You can add new users to the org using either the `signup()` or `create()` methods available on the `UserManager` class. The `signup()` method is limited in scope as it can be used only for adding built-in accounts to an ArcGIS Enterprise instance and not for an org that is hosted on ArcGIS Online. However, you can call the `signup()` anonymously and does not require admin privileges unlike the `create()` method. 
> Note, you can disable self-signup in your ArcGIS Enterprise which would render the `signup()` unusable if you wanted to turn the org invite-only.

You need admin privileges to call the `create()` method. This method is very powerful in an instance of ArcGIS Enterprise, as it allows you to create new accounts from either the arcgis built-in credential store or your enterprise's credential store. For an ArcGIS Online Organization, you can only create users that will use the built-in credential store. For the case of accounts from a built-in credential store, you would provide a password when the account is created. The user can change it at any time once they login. For accounts from your enterprise's credential store, you can ignore the `password` parameter and your users will authenticate through that credential store.

In addition to `role` that can be set, a `level` can be used to allocate accounts based on the privileges that members need. The level determines which privileges are available to the member. The enterprise offers two levels of membership.  Level 1 membership is for members who only need privileges to **view** content, such as maps and apps, that has been shared with them through the organization, as well as join groups within the organization. Level 2 membership is for members who need to view, create, and share content and own groups, in addition to other tasks.

A `user_type` determines the privileges that can be granted to a member. It affects the applications a user can use and actions they can perform in the organization. Learn more about the different values that `user_type` parameter can take [here](https://developers.arcgis.com/rest/enterprise-administration/portal/create-user.htm).


Let us log in to an ArcGIS Enterprise and create some users:

In [ ]:
# let us create a built-in account with username: demo_user1 with org_user privilege
demo_user1 = gis.users.create(username = 'demo_user1',
                              password = '0286eb9ac01f',
                              firstname = 'demo',
                              lastname = 'user',
                              email = 'python@esri.com',
                              description = 'Demonstrating how to create users using ArcGIS Python API',
                              role = 'org_user',
                              level = 2,
                              user_type = 'creatorUT',
                              provider = 'arcgis')

In [ ]:
demo_user1

> Note that we specified `arcgis` as the `provider` argument. If you were creating accounts from your enterprise credential store, you would specify this value as `enterprise` and use the `idpUsername` parameter to specify the username of the user in that credential store. To learn more about this configuration, refer to this help topic on [setting up enterprise logins](https://enterprise.arcgis.com/en/portal/latest/administer/windows/about-configuring-portal-authentication.htm#ESRI_SECTION1_83F7B85FEF594A6B96997AF3CADF3D38).

Note, the `role` parameter was specified as `org_user`. This takes us to the next section on `Role` and `RoleManager` objects.


### Tentang user roles
ArcGIS provides a security concept called roles which defines the privileges a user has within an organization. By default, your org has 3 roles - `org_user`, `org_publisher` and `org_admin`. You can refer to [this topic on organizational roles](https://doc.arcgis.com/en/arcgis-online/reference/roles.htm) to learn about these three roles and their privileges. In summary, a user role can be an active user of the org, create items, join groups and share content. A publisher role has all of user privileges and can create hosted content and perform analysis. An administrator role has all possible privileges. 

Depending on the size of your org and the security needs, you can customize this and create any number of roles with fine grained privileges. For reference on custom roles in an org, refer to [this doc](https://doc.arcgis.com/en/arcgis-online/reference/roles.htm#ESRI_SECTION1_7071F89DE04B448CA833A4164A98DF94)

To know about the role of a `User` object, you can query the `role` property:

In [ ]:
demo_user1_role = demo_user1.role
print(type(demo_user1_role))
print(demo_user1_role)

Since this user was created with a built in role specified as a string, we get back a string with value `org_user`.


### Manajemen role pengguna
Let us create a new role that can only publish tile layers. This role should have none of admin privileges and can have only some of user privileges, namely creating new items and joining groups. 


In [ ]:
# create a tiles publisher role
privilege_list = ['portal:publisher:publishTiles',
                 'portal:user:createItem',
                 'portal:user:joinGroup']

tiles_pub_role = gis.users.roles.create(name = 'tiles_publisher',
                                       description = 'User that can publish tile layers',
                                       privileges = privilege_list)

tiles_pub_role

In [ ]:
# inspect the privileges of this role
tiles_pub_role.privileges

**Note**: the `privileges` parameter was provided a list of strings specifying each individual privilege. Refer to the [api ref doc on the `privileges` parameter](https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.gis.toc.html#arcgis.gis.Role.privileges) to know about the finite list of strings you can use.

In [ ]:
tiles_pub_user = gis.users.create(username='tiles_publisher',
                                 password = 'b0cb0c9f63e',
                                 firstname = 'tiles',
                                 lastname = 'publisher',
                                 email = 'python@esri.com',
                                 description = 'custom role, can only publish tile layers',
                                 role = 'org_user') #org_user as thats the closest.

tiles_pub_user

Querying the `privileges` property of a `User` object returns a list of strings with fine grained privileges. When creating a `Role` object, you can pick and choose from this or refer to the [api ref doc](https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.gis.toc.html#arcgis.gis.Role.privileges).

In [ ]:
tiles_pub_user.privileges

Let us update this user's privileges

In [ ]:
tiles_pub_user.update_role(role = tiles_pub_role)

In [ ]:
# query the privileges to confirm
tiles_pub_user.privileges

Querying the `roleId` property of a `User` returns you the custom roles' ID. You can use this to search for that role to know more details or create another user with the same role:

In [ ]:
tiles_pub_user.roleId

In [ ]:
searched_role = gis.users.roles.get_role(tiles_pub_user.roleId)
searched_role.description

#### Menambah Custom Role

In [ ]:
gis.users.roles.all(max_roles=50)

<a id = "deleting-user-accounts"></a>
## Menghapus akun pengguna
You can delete user accounts by calling the `delete()` method on a `User` object from an account that has administrator privileges. However, deleting raises important questions such as what happens to the content owned by that user? Further, ArcGIS does not allow you to delete users until you have dealt with that users' items and groups. Thus as an administrator, it becomes useful to list and view the content owned by any user in your org.


In [ ]:
# let us access an account named publisher1
publisher1 = gis.users.get('publisher1')
publisher1

In [ ]:
#list all folders as dictionaries
publisher1_folder_list = publisher1.folders
publisher1_folder_list

In [ ]:
# list all items belonging to this user
publisher1_item_list_rootfolder = publisher1.items()
print("Total number of items in root folder: " + str(len(publisher1_item_list_rootfolder)))

#access the first item for a sample
publisher1_item_list_rootfolder[0]

In [6]:
# list all items in the first folder
publisher1.items(folder = publisher1_folder_list[0])


NameError: name 'publisher1' is not defined

Thus using a `GIS` object created with an account that has admin privileges, you were able to query the contents of another user without knowing that user's password or logging in as that user.


### Mendaftarkan ulang user content

As an administrator, you have the privileges to list and view other users' content. When the time comes to delete a user account, you can filter these items and choose to preserve some of them and delete the rest.

Let us delete the `tiles_pub_user` account we created earlier in this guide.

In [ ]:
# list the items owned by the user
tiles_pub_user_items = tiles_pub_user.items()
tiles_pub_user_items

You can reassign specific items to another user by calling the `reassign_to()` method on that `Item` object. Let us reassign the tile layer named `Transport_tiles` to `publisher1` account from earlier. We can get rid of the redundant ocean_tiles items and reassign the rest, to the account `arcgis_python_api`. Since this user does not have privilege to create groups, we do not have to worry about that. We can then delete this user safely.

In [ ]:
# reassign Transport_tiles to publisher1
transport_tiles_item = tiles_pub_user_items[2]
transport_tiles_item

In [ ]:
# the reassign_to() method accepts user name as a string. We can also specify a destination folder name
transport_tiles_item.reassign_to(target_owner = 'publisher1', target_folder= 'f1_english')

In [ ]:
# now let us get rid of redundant ocean tiles items
tiles_pub_user_items[1].delete()

In [ ]:
tiles_pub_user_items[-1].delete()  # an index of -1 in a list refers to the last item

Now we are left with a few more items which should all go to user `arcgis_python_api`. We can either call `reassign_to()` method of the `User` object or call the `delete()` method of the `User` object and pass this information to the `reassign_to` parameter. Let's do that:

In [ ]:
tiles_pub_user.delete(reassign_to='arcgis_python_api')

Thus, we have successfully deleted a user after taking care of that user's content.